In [1]:
import os
import numpy as np
import torch
from torch import tensor
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable, grad

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy
import scipy.io as io
from pyDOE import lhs

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
data = io.loadmat('data/burgers_shock.mat')

N = 2000

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]              

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)

idx = np.random.choice(X_star.shape[0], N, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx,:]

In [3]:
class Network(nn.Module):
    def __init__(self, model):
        super(Network, self).__init__()
        self.model = model
        self.model.apply(self.xavier_init)
        self.lambda_1 = torch.nn.Parameter(torch.tensor([0.0]))
        self.lambda_2 = torch.nn.Parameter(torch.tensor([-6.0]))
        
    def xavier_init(self, m):
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, x, t):
        return self.model(torch.cat([x, t], dim=1))
    
    def loss(self, x, t, y_input):
        lambda_1 = self.lambda_1
        lambda_2 = torch.exp(self.lambda_2)
        
        uf = self.forward(x, t)
        
        # PDE Loss calculation
        u_t = self.gradients(uf, t)[0]
        u_x = self.gradients(uf, x)[0]
        u_xx = self.gradients(u_x, x)[0]
        l_eq = (u_t + lambda_1*uf*u_x - lambda_2*u_xx)
        l_eq = (l_eq**2).mean()
        
        # Loss on the boundary condition
        mse = F.mse_loss(uf, y_input, reduction='mean')
        
        return l_eq + mse
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))

In [4]:
hidden_nodes = 50

model = nn.Sequential(nn.Linear(2, hidden_nodes), 
                        nn.Tanh(), 
                        nn.Linear(hidden_nodes, hidden_nodes),
                        nn.Tanh(), 
                        nn.Linear(hidden_nodes, hidden_nodes),
                        nn.Tanh(), 
                        nn.Linear(hidden_nodes, hidden_nodes),
                        nn.Tanh(),
                        nn.Linear(hidden_nodes, 1))

network = Network(model=model)

In [5]:
X_u_train = tensor(X_u_train).float().requires_grad_(True)
u_train = tensor(u_train).float().requires_grad_(True)

X_star = tensor(X_star).float().requires_grad_(True)
u_star = tensor(u_star).float().requires_grad_(True)

In [6]:
# optimizer = torch.optim.Adam(network.parameters(), lr=3e-4)  # metaopt also has .parameters()
optimizer = torch.optim.LBFGS(network.parameters(), lr=5e-2, max_iter=50, max_eval=50, line_search_fn='strong_wolfe')

best_train_loss = 1e6; epochs = 600
network.train()

weights_path = './saved_path_inverse_burger/pinn.pth'

In [7]:
for i in range(epochs):    
    ### Add the closure function to calculate the gradient. For LBFGS.
    def closure():
        if torch.is_grad_enabled():
            optimizer.zero_grad()
        l = network.loss(X_u_train[:, 0:1], X_u_train[:, 1:2], u_train)
        if l.requires_grad:
            l.backward()
        return l

    optimizer.step(closure)

    # calculate the loss again for monitoring
    l = closure()

    if i > 400 and float(l.item()) < best_train_loss:
        torch.save(network.state_dict(), weights_path)
        best_train_loss = float(l.item())

    if (i % 100) == 0:
        print("Epoch {}: ".format(i), l.item())

Epoch 0:  0.04405350238084793
Epoch 100:  2.3927972506498918e-05
Epoch 200:  1.0691312127164565e-05
Epoch 300:  1.0691312127164565e-05
Epoch 400:  1.0691312127164565e-05
Epoch 500:  1.0691312127164565e-05


In [8]:
### Loading the best weights ###
network.load_state_dict(torch.load(weights_path))

<All keys matched successfully>

In [9]:
network.eval()

Network(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Tanh()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Tanh()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Tanh()
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Tanh()
    (8): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [10]:
nu = 0.01 / np.pi

error_lambda_1 = np.abs(network.lambda_1.detach().item() - 1.0)*100
error_lambda_2 = np.abs(torch.exp(network.lambda_2).detach().item() - nu) / nu * 100

error_lambda_1, error_lambda_2

(0.1516401767730713, 2.289382591936315)

In [11]:
1.0, network.lambda_1.detach().item()

(1.0, 0.9984835982322693)

In [12]:
nu, torch.exp(network.lambda_2).detach().item()

(0.003183098861837907, 0.003255972173064947)